In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [25]:
model = VGG16(weights = 'imagenet', include_top = False, pooling  = 'max', input_shape = (48, 48, 3))
# model = VGG16(weights = 'imagenet', include_top = False, input_shape = (48, 48, 3))

In [3]:
def processString(s):
    s = s.split()
    img = np.zeros((48, 48))
    for i in range(48):
        for j in range(48):
            img[i][j] = s[48*i + j]
    img = np.repeat(img[:, :, np.newaxis], 3, axis = 2)
    return img


def getDataset():
    data_path = '../Data/fer2013/fer2013.csv'
    df = pd.read_csv(data_path)
    X_train = []; y_train = []
    X_test = []; y_test = []
    X_val = []; y_val = []
    for index, row in df.iterrows():
        if(row['Usage'] == 'Training'):
            if(len(X_train) < 1e+4):
                X_train.append(processString(row['pixels']))
                y_train.append(row['emotion'])
        elif(row['Usage'] == 'PublicTest'):
            X_val.append(processString(row['pixels']))
            y_val.append(row['emotion'])
        else:
            X_test.append(processString(row['pixels']))
            y_test.append(row['emotion'])
    
    #preprocess the data
    mean = np.mean(X_train, axis = 0)
    X_train -= mean
    X_test -= mean
    X_val -= mean
    data = {}
    data['X_train'] = np.array(X_train); data['y_train'] = np.array(y_train)
    data['X_val'] = np.array(X_val); data['y_val'] = np.array(y_val)
    data['X_test'] = np.array(X_test); data['y_test'] = np.array(y_test)
    return data

In [15]:
data = getDataset()
# import pickle
# f = open('../Data/data.pickle', 'rb')
# data = pickle.load(f)

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

In [26]:
for layer in model.layers:
    layer.trainable = False

In [27]:
x = model.output
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.25)(x)
x = Dense(256, activation = 'relu')(x)
predictions = Dense(7, activation = 'softmax')(x)

model_final = Model(input = model.input, output = predictions)

sgd = SGD(lr = 0.001, decay = 1e-5, momentum = 0.9, nesterov = True)
model_final.compile(loss = 'categorical_crossentropy', 
                    optimizer = 'adam', metrics = ['accuracy'])

E:\Study-Ebooks\Sem-7\Anaconda\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  import sys


In [18]:
data['X_train'].shape

(10000, 48, 48, 3)

In [20]:
# X_train = data['X_train'].reshape(data['X_train'].shape[0], 48, 48, 3)
# X_test = data['X_test'].reshape(data['X_test'].shape[0], 48, 48, 3)
# X_val = data['X_val'].reshape(data['X_val'].shape[0], 48, 48, 3)
X_train = data['X_train']
X_test = data['X_test']
X_val = data['X_val']
y_train = np_utils.to_categorical(data['y_train'])
y_test = np_utils.to_categorical(data['y_test'])
y_val = np_utils.to_categorical(data['y_val'])

In [28]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [ ]:
model_final.fit(X_train, y_train, validation_data = (X_val, y_val), batch_size = 32, epochs = 6)

Train on 10000 samples, validate on 3589 samples
Epoch 1/6
10000/10000 [==============================] - 380s 38ms/step - loss: 3.3691 - acc: 0.2711 - val_loss: 1.6872 - val_acc: 0.3288
Epoch 2/6
 5408/10000 [===============>..............] - ETA: 43:20 - loss: 1.6815 - acc: 0.3323

In [100]:
score = model_final.evaluate(X_test, y_test)

3534/3534 [==============================] - 112s 32ms/step


In [101]:
score

[1.5640899492106197, 0.3766270515165831]

In [34]:
unique,counts = np.unique(y_train, return_counts = True)

In [35]:
for u, c in zip(unique,counts):
    print(u,c)

0 1407
1 160
2 1431
3 2537
4 1663
5 1091
6 1711


In [33]:
y_train = data['y_train']

In [93]:
X_train = data['X_train']
y_train = data['y_train']
X_test = data['X_test']
y_test = data['y_test']
X_val = data['X_val']
y_val = data['y_val']

In [94]:
scale = {0:0, 1:-1, 2:1, 3:2, 4:3, 5:4, 6:5}

In [95]:
X_train = np.array([X_train[i] for i in range(len(X_train)) if y_train[i] != 1])
X_test = np.array([X_test[i] for i in range(len(X_test)) if y_test[i] != 1])
X_val = np.array([X_val[i] for i in range(len(X_val)) if y_val[i] != 1])
y_train = np.array([scale[y] for y in y_train if y != 1])
y_test = np.array([scale[y] for y in y_test if y != 1])
y_val = np.array([scale[y] for y in y_val if y != 1])

In [97]:
X_val.shape

(3533, 48, 48, 3)